Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/using-mlflow/deploy-model/deploy-model.png)

## Deploy Model as Azure Machine Learning Web Service using MLflow

This example shows you how to use mlflow together with Azure Machine Learning services for deploying a model as a web service. You'll learn how to:

 1. Retrieve a previously trained scikit-learn model
 2. Create a Docker image from the model
 3. Deploy the model as a web service on Azure Container Instance
 4. Make a scoring request against the web service.

## Prerequisites and Set-up

This notebook requires you to first complete the [Use MLflow with Azure Machine Learning for Local Training Run](../train-local/train-local.ipnyb) or [Use MLflow with Azure Machine Learning for Remote Training Run](../train-remote/train-remote.ipnyb) notebook, so as to have an experiment run with uploaded model in your Azure Machine Learning Workspace.

Also install following packages if you haven't already

```
pip install azureml-mlflow pandas
```

Then, import necessary packages:

In [ ]:
import mlflow
import azureml.mlflow
import azureml.core
from azureml.core import Workspace

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

### Connect to workspace and set MLflow tracking URI

Setting the tracking URI is required for retrieving the model and creating an image using the MLflow APIs.

In [ ]:
ws = Workspace.from_config()

mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())

### Retrieve model from previous run

Let's retrieve the experiment from training notebook, and list the runs within that experiment.

In [ ]:
experiment_name = "experiment-with-mlflow"
exp = ws.experiments[experiment_name]

runs = list(exp.get_runs())
runs

Then, let's select the most recent training run and find its ID. You also need to specify the path in run history where the model was saved. 

In [ ]:
runid = runs[0].id
model_save_path = "model"

### Create Docker image

To create a Docker image with Azure Machine Learning for Model Management, use ```mlflow.azureml.build_image``` method. Specify the model path, your workspace, run ID and other parameters.

MLflow automatically recognizes the model framework as scikit-learn, and creates the scoring logic and includes library dependencies for you.

Note that the image creation can take several minutes.

In [ ]:
import mlflow.azureml

azure_image, azure_model = mlflow.azureml.build_image(model_uri="runs:/{}/{}".format(runid, model_save_path),
                                                      workspace=ws,
                                                      model_name='diabetes-sklearn-model',
                                                      image_name='diabetes-sklearn-image',
                                                      synchronous=True)

### Deploy web service

Let's use Azure Machine Learning SDK to deploy the image as a web service. 

First, specify the deployment configuration. Azure Container Instance is a suitable choice for a quick dev-test deployment, while Azure Kubernetes Service is suitable for scalable production deployments.

Then, deploy the image using Azure Machine Learning SDK's ```deploy_from_image``` method.

Note that the deployment can take several minutes.

In [ ]:
from azureml.core.webservice import AciWebservice, Webservice


aci_config = AciWebservice.deploy_configuration(cpu_cores=1, 
                                                memory_gb=1, 
                                                tags={"method" : "sklearn"}, 
                                                description='Diabetes model',
                                                location='eastus2')


# Deploy the image to Azure Container Instances (ACI) for real-time serving
webservice = Webservice.deploy_from_image(
    image=azure_image, workspace=ws, name="diabetes-model-1", deployment_config=aci_config)


webservice.wait_for_deployment(show_output=True)

## Make a scoring request

Let's take the first few rows of test data and score them using the web service

In [ ]:
test_rows = [
    [0.01991321,  0.05068012,  0.10480869,  0.07007254, -0.03596778,
     -0.0266789 , -0.02499266, -0.00259226,  0.00371174,  0.04034337],
    [-0.01277963, -0.04464164,  0.06061839,  0.05285819,  0.04796534,
     0.02937467, -0.01762938,  0.03430886,  0.0702113 ,  0.00720652],
    [ 0.03807591,  0.05068012,  0.00888341,  0.04252958, -0.04284755,
     -0.02104223, -0.03971921, -0.00259226, -0.01811827,  0.00720652]]

MLflow-based web service for scikit-learn model requires the data to be converted to Pandas DataFrame, and then serialized as JSON. 

In [ ]:
import json
import pandas as pd

test_rows_as_json = pd.DataFrame(test_rows).to_json(orient="split")

Let's pass the conveted and serialized data to web service to get the predictions.

In [ ]:
predictions = webservice.run(test_rows_as_json)

print(predictions)

You can use the web service's scoring URI to make a raw HTTP request

In [ ]:
webservice.scoring_uri

You can diagnose the web service using ```get_logs``` method.

In [ ]:
webservice.get_logs()

## Next Steps

Learn about [model management and inference in Azure Machine Learning service](https://docs.microsoft.com/en-us/azure/machine-learning/service/concept-model-management-and-deployment).